In [36]:
!pip install langchain_community

In [37]:
import os
from dotenv import load_dotenv
load_dotenv()

gorq_api_key=os.getenv("GROQ_TESTING_API")


In [38]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=gorq_api_key)

In [39]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(content="hi my name is Sudesh Patil"),
        AIMessage(content="Hello Sudesh Patil, it's nice to meet you! 👋\n\nIs there anything I can help you with today?\n"),
        HumanMessage(content="what's my name?")

    ]
)

AIMessage(content='Your name is Sudesh Patil.\n\nI remember that you told me earlier! 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 58, 'total_tokens': 90, 'completion_time': 0.058181818, 'prompt_time': 0.003449243, 'queue_time': 0.235166871, 'total_time': 0.061631061}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f8b6dd09-df0e-4f0e-8a5e-ea59164434d9-0', usage_metadata={'input_tokens': 58, 'output_tokens': 32, 'total_tokens': 90})

### Message History
it is concept in langchian which stores the past coversation and use it in further future conversation happend related to that past conversation.

In [40]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_messesge_history=RunnableWithMessageHistory(model, get_session_history)

In [41]:
config={"configurable":{"session_id":"chat"}}

In [42]:
responce=with_messesge_history.invoke(
    [HumanMessage(content="hey, my name is Sudesh")],
    config=config
)

In [43]:
responce.content

"Hi Sudesh, it's nice to meet you! 👋\n\nWhat can I do for you today? 😄  \n\n"

In [44]:
with_messesge_history.invoke(
    [HumanMessage(content="hey what,s my name?")],
    config=config
)

AIMessage(content='Your name is Sudesh!  😊 \n\nI remember that from when you first introduced yourself.  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 58, 'total_tokens': 93, 'completion_time': 0.063636364, 'prompt_time': 0.003602824, 'queue_time': 0.237785837, 'total_time': 0.067239188}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf591de5-fcd1-4f7c-b8a6-13edaa183bb2-0', usage_metadata={'input_tokens': 58, 'output_tokens': 35, 'total_tokens': 93})

In [45]:
config1={"configurable":{"session_id":"chat2"}}
responce=with_messesge_history.invoke(
    [HumanMessage(content="hey, my name is Sudesh")],
    config=config1
)
responce.content

"Hello Sudesh, it's nice to meet you! 👋\n\nHow can I help you today? 😊\n"

### Prompt template
it is kind of thing that converts raw message to message that llm can easyly undersatandable and use and give response as per input message let us see how it works?

In [46]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder #this ti mostly used in assigning propts such  that it will best for messages

prompts=ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful  assisitent.Answer all question to the best of your ability"),
        MessagesPlaceholder(variable_name="message")
    ]
) 

chain=prompts|model



In [47]:
chain.invoke({"message":[HumanMessage(content="Hi my name is sudesh")]})

AIMessage(content="Hello Sudesh! It's nice to meet you. \n\nI'm ready to answer any questions you have to the best of my ability. Ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.002366336, 'queue_time': 0.240085584, 'total_time': 0.075093609}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-032e9b5f-b1da-48fa-91a6-1a669e607c24-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71})

In [48]:
with_messesge_history=RunnableWithMessageHistory(chain, get_session_history)

In [49]:
config={"configurable":{"session_id":"chat3"}}
responce=with_messesge_history.invoke(
    [HumanMessage(content="Hi my name is sudesh")],
    config=config
)
responce.content

"Hello Sudesh, it's nice to meet you! \n\nI'm happy to be your helpful assistant.  What can I do for you today? 😊  Just ask away! \n"

In [50]:
# as it is very simple let us add more complexityes to it

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder #this ti mostly used in assigning propts such  that it will best for messages

prompts=ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful  assisitent.Answer all question to the best on {language}"),
        MessagesPlaceholder(variable_name="message")
    ]
) 

chain=prompts|model


In [51]:
chain.invoke({"message":[HumanMessage(content="Hi my name is sudesh")],"language":"Marathi"})

AIMessage(content='नाही, मी तुमच्या मदतीसाठी येथे आहे, सुदेश! कोणत्याही प्रश्नांसाठी मला विचारू नको. मला तुमच्या मदत करण्यास आनंद होईल.  😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 30, 'total_tokens': 85, 'completion_time': 0.1, 'prompt_time': 0.002164487, 'queue_time': 0.232304383, 'total_time': 0.102164487}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-692678c8-0c7f-487b-8eaf-e1e1aa3d22fe-0', usage_metadata={'input_tokens': 30, 'output_tokens': 55, 'total_tokens': 85})

In [52]:
# now we sould warp the contennt in morw complicated way
with_messesge_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="message"
)

In [53]:
config={"configurable":{"session_id":"chat4"}}
response=with_messesge_history.invoke(
    {'message':[HumanMessage(content="Hi, I am Sudesh")], "language":"Marathi"},
    config=config
)

responce.content

"Hello Sudesh, it's nice to meet you! \n\nI'm happy to be your helpful assistant.  What can I do for you today? 😊  Just ask away! \n"

In [54]:
from langchain_core.messages import SystemMessage, HumanMessage, trim_messages

# trimmer is the concept used for save the history of past conversatio  such that chatbot will use the history for future conversation 
trimmer=trim_messages(
     max_tokens=45,
     strategy="last",
     token_counter=model,
     include_system=True,
     allow_partial=False,
     start_on="human"
 )

message= [
    SystemMessage(content="you're a good ai assistant"),
    HumanMessage(content="hi! i am cheti cool"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like venilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(message)

[SystemMessage(content="you're a good ai assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [55]:
!pip install transformers

In [56]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(message=itemgetter("message")|trimmer)
    | prompts
    | model
)

respons=chain.invoke(
    {
        "message": message + [HumanMessage(content="What ice cream i like ?")],
        "language":"English"
    }
)

responce.content

"Hello Sudesh, it's nice to meet you! \n\nI'm happy to be your helpful assistant.  What can I do for you today? 😊  Just ask away! \n"

In [57]:
respons=chain.invoke(
    {
        "message": message + [HumanMessage(content="What problem i asked?")],
        "language":"English"
    }
)


In [58]:
responce.content

"Hello Sudesh, it's nice to meet you! \n\nI'm happy to be your helpful assistant.  What can I do for you today? 😊  Just ask away! \n"

In [59]:
# now we wrap the message
with_wrap_message=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="message",
    
    
)

config={"configurable":{"session_id":"chat5"}}

In [60]:
respons=with_messesge_history.invoke(
    {
        "message": [HumanMessage(content="What ice cream i like ?")],
        "language":"English"
    },
    config=config,
)

responce.content

"Hello Sudesh, it's nice to meet you! \n\nI'm happy to be your helpful assistant.  What can I do for you today? 😊  Just ask away! \n"